In [111]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/iris/database.sqlite
/kaggle/input/iris/Iris.csv


In [112]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
df=pd.read_csv("/kaggle/input/iris/Iris.csv")
df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [113]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             150 non-null    int64  
 1   SepalLengthCm  150 non-null    float64
 2   SepalWidthCm   150 non-null    float64
 3   PetalLengthCm  150 non-null    float64
 4   PetalWidthCm   150 non-null    float64
 5   Species        150 non-null    object 
dtypes: float64(4), int64(1), object(1)
memory usage: 7.2+ KB


In [114]:
df.describe()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,75.500000,5.843333,3.054000,3.758667,1.198667
std,43.445368,0.828066,0.433594,1.764420,0.763161
min,1.000000,4.300000,2.000000,1.000000,0.100000
25%,38.250000,5.100000,2.800000,1.600000,0.300000
50%,75.500000,5.800000,3.000000,4.350000,1.300000
75%,112.750000,6.400000,3.300000,5.100000,1.800000
max,150.000000,7.900000,4.400000,6.900000,2.500000


In [115]:
X = df[['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']]

In [116]:
def assign(x):
    if x=='Iris-setosa':
        return 1
    if x=='Iris-versicolor':
        return 2
    if x=='Iris-virginica':
        return 3
    
y_in = df['Species'].apply(assign)

In [117]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_in, test_size=0.4)
X_train, X_test, y_train, y_test = X_train.T, X_test.T, y_train.values.reshape(1,y_train.shape[0]), y_test.values.reshape(1,y_test.shape[0])

In [118]:
def layer_sizes(X, y):
    n_x = X.shape[0]
    n_h = 5
    n_y = y.shape[0]
    
    return n_x, n_h, n_y

In [119]:
def initialize_parameters(n_x, n_h, n_y):
    W1 = np.random.randn(n_h, n_x)*0.01
    b1 = np.zeros((n_h, 1))
    W2 = np.random.randn(n_y, n_h)*0.01
    b2 = np.zeros((n_y, 1))
    
    parameters = {'W1':W1,
                 'b1':b1,
                 'W2':W2,
                 'b2':b2}
    
    return parameters

In [120]:
def sigmoid(z):
    s = 1/(1+np.exp(-z))
    return s

def forward_propagation(X, parameters):
    W1=parameters['W1']
    b1=parameters['b1']
    W2=parameters['W2']    
    b2=parameters['b2']
    
    Z1 = np.dot(W1, X)+b1
    A1 = np.tanh(Z1)
    Z2 = np.dot(W2, A1)+b2
    A2 = Z2
    
    cache = {'Z1':Z1,
            'A1':A1,
            'Z2':Z2,
            'A2':A2}
    
    return A2, cache
    

In [121]:
def compute_cost(A2, Y, parameters):
    m = Y.shape[1]
    
    logprobs = Y*np.log(A2)+(1-Y)*np.log(1-A2)
    cost = -np.sum(logprobs)/m
    
    return cost

In [122]:
def backward_propagation(parameters, cache, X, Y):
    m = X.shape[1]
    
    W1 = parameters['W1']
    W2 = parameters['W2']
    
    A1 = cache['A1']
    A2 = cache['A2']
    
    dZ2 = A2-Y
    dW2 = np.dot(dZ2, A1.T)/m
    db2 = np.sum(dZ2, axis=1, keepdims=True)/m
    dZ1 = np.dot(W2.T, dZ2)*(1-A1**2)
    dW1 = np.dot(dZ1, X.T)/m
    db1 = np.sum(dZ1, axis=1, keepdims=True)/m
    
    grads = {'dW1':dW1,
            'db1':db1,
            'dW2':dW2,
            'db2':db2}
    
    return grads

In [123]:
def update_parameters(parameters, grads, alpha=0.009):
    W1=parameters['W1']
    b1=parameters['b1']    
    W2=parameters['W2']
    b2=parameters['b2']
    
    dW1=grads['dW1']
    db1=grads['db1']
    dW2=grads['dW2']
    db2=grads['db2']
    
    W1 = W1-alpha*dW1
    b1 = b1-alpha*db1
    W2 = W2-alpha*dW2
    b2 = b2-alpha*db2
    
    parameters={'W1':W1,
               'b1':b1,
               'W2':W2,
               'b2':b2}
    
    return parameters

In [124]:
def nn_model(X, Y, n_h, num_iterations = 10000, print_cost=False):
    n_x=layer_sizes(X, Y)[0]
    n_y=layer_sizes(X, Y)[2]
    
    parameters = initialize_parameters(n_x, n_h, n_y)
    
#     grdient descent
    for i in range(0, num_iterations):
        A2, cache = forward_propagation(X, parameters)
        
        cost = compute_cost(A2, Y, parameters)
        
        grads = backward_propagation(parameters, cache, X, Y)
        
        parameters = update_parameters(parameters, grads)
        
        if print_cost and i%1000==0:
            print(f'Cost after iteration {i} is {cost}')
        
    return parameters

In [125]:
def predict(parameters, X):
    A2, cache = forward_propagation(X, parameters)
    predictions = np.round(A2).astype(int)
    
    return predictions

In [126]:
parameters = nn_model(X_train, y_train, n_h=4,
                     num_iterations=50000, print_cost=False)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in log
  after removing the cwd from sys.path.


In [127]:
predictions = predict(parameters, X_test)

In [128]:
from sklearn.metrics import accuracy_score
out = pd.DataFrame({'real':y_test[0], 'predicted':predictions[0]})

In [129]:
score = accuracy_score(out['real'], out['predicted'])
print(score*100)

93.33333333333333
